---
## `Dummy Data`
---

In [1]:
# P = range(len(banker))
# B = range(len(bank))
# L = range(len(loan))

# T = range(len(loan_type))

# type = ["living", "investment"]
# every banker has capacity & salary, 2 list of size 
## banker[p][1] gives salary of p-th banker 
# every bank has capacity, 1 list of size M 
## bank[b][0] gives cap of b-th banker 
# loan 
# dist[p][b] dist. betw. banker p & bank b 

---
## `Test Data (Smaller)`
---

In [8]:
import pandas as pd
import math

banker = pd.read_excel('.//banker_data//banker_data.xlsx')
bank = pd.read_csv('.//bank_loan_data//bank.csv')
loan = pd.read_csv('.//bank_loan_data//loan.csv')
loan_type = loan['loan_type'].unique()   # loan_type = [0,1]

P = range(round(len(banker)/4))
B = range(round(len(bank)/4))
L = range(round(len(loan)/30))
T = range(len(loan_type))
print([P,B,L,T])
        
# yoinked from ass1 
dist = [ [0]*len(B) for i in P]
for p in P:
     for b in B:
        # LA,LONG for banker is column 1,2; for banks LONG,LA is 1:2
        dist[p][b] = math.dist(banker.iloc[p, 1:3],bank.iloc[b, 2:0:-1])


print("bank\n", bank.head())
print("loan\n",loan.head())
print("banker\n",banker.head())

[range(0, 88), range(0, 42), range(0, 703), range(0, 2)]
bank
                                             location        long        lat  \
0  Westfield Fountain Gate, 352 Princes Highway, ...  145.304441 -38.018206   
1               330 Collins St, Melbourne, VIC, 3000  144.963217 -37.816309   
2               460 Collins St, Melbourne, VIC, 3000  144.958852 -37.817673   
3               288 Lygon Street, Carlton, VIC, 3053  144.967263 -37.800216   
4                  22 Watton St, Werribee, VIC, 3030  144.661003 -37.901344   

      capacity  
0   91908072.0  
1  132948034.0  
2  137102661.0  
3  227108391.0  
4   77150960.0  
loan
    loan_amount  loan_type
0    1590071.0          1
1    1589968.0          1
2    1589900.0          0
3    1590024.0          1
4    1589990.0          1
banker
    banker_id   Latitude   Longitude  tenure (in years)  capacity  v1_rand_0_1  \
0          1 -37.695528  143.951218           8.183687      1000     0.863803   
1          2 -37.551700  144

---
## `Data`
---

In [3]:
import pandas as pd
import math

banker = pd.read_excel('.//banker_data//banker_data.xlsx')
bank = pd.read_csv('.//bank_loan_data//bank.csv')
loan = pd.read_csv('.//bank_loan_data//loan.csv')
loan_type = loan['loan_type'].unique()   # loan_type = [0,1] 
# type = ["living", "investment"]

P = range(len(banker))
B = range(len(bank))
L = range(len(loan))
T = range(len(loan_type))
print([P,B,L,T])
        
# yoinked from ass1 
dist = [ [0]*len(B) for i in P]
for p in P:
     for b in B:
        # LA,LONG for banker is column 1,2; for banks LONG,LA is 1:2
        dist[p][b] = math.dist(banker.iloc[p, 1:3],bank.iloc[b, 2:0:-1])


print("bank\n", bank.head())
print("loan\n",loan.head())
print("banker\n",banker.head())

[range(0, 350), range(0, 169), range(0, 21095), range(0, 2)]
bank
                                             location        long        lat  \
0  Westfield Fountain Gate, 352 Princes Highway, ...  145.304441 -38.018206   
1               330 Collins St, Melbourne, VIC, 3000  144.963217 -37.816309   
2               460 Collins St, Melbourne, VIC, 3000  144.958852 -37.817673   
3               288 Lygon Street, Carlton, VIC, 3053  144.967263 -37.800216   
4                  22 Watton St, Werribee, VIC, 3030  144.661003 -37.901344   

      capacity  
0   91908072.0  
1  132948034.0  
2  137102661.0  
3  227108391.0  
4   77150960.0  
loan
    loan_amount  loan_type
0    1590071.0          1
1    1589968.0          1
2    1589900.0          0
3    1590024.0          1
4    1589990.0          1
banker
    banker_id   Latitude   Longitude  tenure (in years)  capacity  v1_rand_0_1  \
0          1 -37.695528  143.951218           8.183687      1000     0.863803   
1          2 -37.551700 

---

## `Model`

---

In [4]:
# [loan.iloc[l] for l in L for t in T if loan.iloc[l]["loan_type"]==t]
# loan.iloc[1]["loan_type"]

# count = 0
# c2 = 0
# for t in T:
#     for l in L:
#         if loan.iloc[l]["loan_type"]==t:
#             print(loan.iloc[l])
#     if t == 20:
#         break
    
# bank.iloc[0]["capacity"]
# banker.iloc[0]["salary (in k)"]
# loan.iloc[0]["loan_amount"]

In [18]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model()
m.Params.OutputFlag = 1

x = m.addVars(T,L,B, name="x", vtype = GRB.BINARY)
y = m.addVars(P,B, name="y", vtype = GRB.BINARY)
z = m.addVars(T,L,P, name="z", vtype = GRB.BINARY)
zx = m.addVars(T,L,P,B, name="zx", vtype = GRB.BINARY) #aux var 
 
# Objective
m.setObjective( gp.quicksum( dist[p][b] for b in B for p in P) +
                gp.quicksum( banker.iloc[0]["salary (in k)"]*y[p,b] for p in P for b in B) , GRB.MINIMIZE )

# Loans with corres. banker # 
# source https://support.gurobi.com/hc/en-us/community/posts/360056771292-Invalid-argument-to-QuadExpr-multiplication-Error-
# https://support.gurobi.com/hc/en-us/community/posts/360072027392-How-to-deal-with-a-product-of-3-different-decision-variables-in-a-term-
# aux constraints 
m.addConstrs( (z[t,l,p]*x[t,l,b]
                == zx[t,l,p,b] for t in T for l in L if loan.iloc[l]["loan_type"]==t  for p in P for b in B),
             name='3a.auxilliary constraint')

m.addConstrs( (gp.quicksum(y[p,b]*zx[t,l,p,b] for p in P)
               <= 1 for t in T for l in L if loan.iloc[l]["loan_type"]==t for b in B),
             name='3.each loan by 1 banker')

## Constraints ##
# Loans # 
m.addConstrs( (gp.quicksum( x[t,l,b] for b in B)
               <= 1 for t in T for l in L ),
             name='1.each loan 1 bank')
m.addConstrs( (gp.quicksum( x[t,l,b]*loan.iloc[l]["loan_amount"] for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
               <= bank.iloc[b]["capacity"] for b in B ), 
             name='2.total loan respect bank cap')

# # Loans with corres. banker # 
# doesnt work, not really sure why, something about multiplying 3 vars
# m.addConstrs( (gp.quicksum(z[t,l,p]*x[t,l,b]*y[p,b] for p in P) 
#                <= 1 for t in T for l in L if loan.iloc[l]["loan_type"]==t for b in B),
#              name='3.each loan by 1 banker')

m.addConstrs( (gp.quicksum( z[t,l,p]*loan.iloc[l]["loan_amount"] for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
               <= banker.iloc[p]["capacity"] for p in P),
             name='4.value of loans respect banker cap')
             
# Bankers # 
m.addConstrs( (gp.quicksum( y[p,b] for b in B) 
               <= 1 for p in P),
             name='5.each banker for 1 bank')

# Restrictions on types of loans # 
m.addConstr( (gp.quicksum( x[1,l,b] * loan.iloc[l]["loan_amount"] for b in B for l in L if loan.iloc[l]["loan_type"]==1) 
             <=gp.quicksum( x[t,l,b] * loan.iloc[l]["loan_amount"] * 0.3 for b in B for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
              ),
             name='6.value of invest loans less than 30% total')
#### for every banker? 

m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 1625 rows, 5383052 columns and 183664 nonzeros
Model fingerprint: 0xdbf30bfb
Model has 2627814 quadratic constraints
Variable types: 0 continuous, 5383052 integer (5383052 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+06]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [9e+01, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+08]
  QRHS range       [1e+00, 1e+00]
Found heuristic solution: objective 2807.1985185

Explored 0 nodes (0 simplex iterations) in 0.87 seconds (0.12 work units)
Thread count was 1 (of 20 available processors)

Solution count 1: 2807.2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.807198518522e+03, best bound 2.807198518522e+03, gap 0.0